In [ ]:
#!/usr/bin/env python
import sys
import os

# change base folder
os.chdir('/mnt/fasttalk/')

# --- Imports & paths ---
from pathlib import Path
import os, subprocess
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from matplotlib import animation
import torch
import cv2
import numpy as np
from flame_model.FLAME import FLAMEModel
from renderer.renderer import Renderer
import argparse
import torch.nn.functional as F
from pytorch3d.transforms import matrix_to_euler_angles


device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
flame    = FLAMEModel(n_shape=300,n_exp=50).to(device)
renderer = Renderer(render_full_head=True).to(device)

# >>> EDIT THESE <<<
sequence_path = "/mnt/smirk/results/npz/6iXGqEq_cpI_0062_S286_E323_L299_T143_R619_B463.npz"
audio_path    = sequence_path.replace("npz", "wav")
output_dir    = "/mnt/fasttalk/demo/samples"

Path(output_dir).mkdir(parents=True, exist_ok=True)
base_name     = Path(sequence_path).stem


In [ ]:
# --- Load NPZ, extract values, smooth pose ---
flame_param = np.load(sequence_path, allow_pickle=True)

for k in flame_param.keys():
    print(f"Key: {k}, Shape: {flame_param[k].shape}")

def extract_expr_jaw_gpose(arr):
    
    expr = arr['exp'].reshape(-1, 50)
    jaw  = arr['jaw'].reshape(-1,3)
    gpose= arr['pose'].reshape(-1, 3)
    shape = arr['mica_shape'].reshape(1, -1)
    eyelid = arr['eyes'].reshape(-1, 2)


    gpose = gpose - gpose.mean(axis=0, keepdims=True)
    return expr, jaw, gpose, shape, eyelid

expr, jaw, gpose, shape, eyelid = extract_expr_jaw_gpose(flame_param)
T = len(expr)
if T >= 7:
    gpose = savgol_filter(gpose, window_length=7, polyorder=2, axis=0)

def get_fps(arr, default=25):
    try: return int(arr.get('fps', default))
    except Exception: return default
fps = get_fps(flame_param, default=25)
print(f"Loaded: T={T}, fps={fps}, audio={'yes' if os.path.exists(audio_path) else 'no'}")


In [ ]:
def get_vertices_from_blendshapes(expr, gpose, jaw, shape, eyelids=None):

    # Load the encoded file
    expr_tensor    = expr.to(device)
    gpose_tensor   = gpose.to(device)
    jaw_tensor     = jaw.to(device)
    
    if eyelids is not None:
        eyelids_tensor = eyelids.to(device)

    target_shape_tensor = shape.expand(expr_tensor.shape[0], -1).to(device)

    I = matrix_to_euler_angles(torch.cat([torch.eye(3)[None]], dim=0),"XYZ").to(device)

    eye_r    = I.clone().to(device).squeeze()
    eye_l    = I.clone().to(device).squeeze()
    eyes     = torch.cat([eye_r,eye_l],dim=0).expand(expr_tensor.shape[0], -1).to(device)

    pose = torch.cat([gpose_tensor, jaw_tensor], dim=-1).to(device)

    flame_output_only_shape,_ = flame.forward(shape_params=target_shape_tensor, 
                                               expression_params=expr_tensor, 
                                               pose_params=pose, 
                                               eye_pose_params=eyes)
    return flame_output_only_shape.detach()

def update(frame_inx, renderer_output_blendshapes, axes):
    # Select the frames to plot
    frame = renderer_output_blendshapes['rendered_img'][frame_inx].detach().cpu().numpy().transpose(1, 2, 0)

    # Update the second subplot
    axes.clear()
    axes.imshow((frame * 255).astype(np.uint8))
    axes.axis('off')
    axes.set_title(f'Frame Stage 1 (Blendshape) {frame_inx + 1}')

In [ ]:
# --- Vertices/Camera (REPLACE with your real computation) ---
## This is a stub placeholder — you must compute the vertices & camera as in your prior pipeline.
## Example names below mirror your message: `blendshapes_derived_vertices`, `cam`.

# raise RuntimeError("Replace this cell with your code that produces blendshapes_derived_vertices and cam.")

# Minimal placeholders so the notebook runs — replace with real data.
H, W = 512, 512
blendshapes_derived_vertices = get_vertices_from_blendshapes(torch.tensor(expr, dtype=torch.float32), 
                                                             torch.tensor(gpose, dtype=torch.float32), 
                                                             torch.tensor(jaw, dtype=torch.float32),
                                                             torch.tensor(shape, dtype=torch.float32),
                                                             torch.tensor(eyelid, dtype=torch.float32))

cam = torch.tensor([5, 0, 0], dtype=torch.float32).unsqueeze(0).to(device)
cam = cam.expand(blendshapes_derived_vertices.shape[0], -1)

In [ ]:
# Render the frames
renderer_output_blendshapes  = renderer.forward(blendshapes_derived_vertices, cam)

N = renderer_output_blendshapes['rendered_img'].shape[0] # Number of frames

# Create a figure with two subplots
fig, axes = plt.subplots(1, 1, figsize=(5, 5))

# Create an animation
ani = animation.FuncAnimation(
    fig, 
    update, 
    frames=N, 
    fargs=(renderer_output_blendshapes, axes),
    interval=100
)

# Save the animation as a video file
video_file = f'{output_dir}/{base_name}.mp4'
ani.save(video_file, writer='ffmpeg', fps=25)
print(f"Video saved as {video_file}")


In [ ]:
# =============== Add audio to the video ===============

# Add audio to the video
output_with_audio = f'{output_dir}/{base_name}_with_audio.mp4'
if os.path.exists(audio_path):
    cmd = f'ffmpeg -y -i {video_file} -i {audio_path} -c:v copy -c:a aac -strict experimental {output_with_audio}'
    subprocess.run(cmd, shell=True)
    print(f"Video with audio saved as {output_with_audio}")
else:
    print(f"Audio file {audio_path} not found")